In [2]:
%pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import json
import time
from tqdm.auto import tqdm

import numpy as np
import pandas as pd

import torch

from datasets import load_dataset

In [4]:
tqdm.pandas()

In [5]:
NUM_EXPERIMENTS = 10
# MODEL_NAME = "{}-s{}-bs{}-e{}-i{}-h{}"
MODEL_NAME = "{}-i{}-h{}"
MODEL_PATH = "../models/{}/{}"

In [6]:
langs = ["java", "python", "pharo"]

labels = {
    "java": ["summary", "Ownership", "Expand", "usage", "Pointer", "deprecation", "rational"],
    "python": ["Usage", "Parameters", "DevelopmentNotes", "Expand", "Summary"],
    "pharo": ["Keyimplementationpoints", "Example", "Responsibilities", "Classreferences", "Intent", "Keymessages", "Collaborators"]
}

In [7]:
alias = "nb"
num_iterations = 0
head = "NONE"

In [8]:
dataset = load_dataset("NLBSE/nlbse25-code-comment-classification")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

java_train-00000-of-00001.parquet:   0%|          | 0.00/680k [00:00<?, ?B/s]

java_test-00000-of-00001.parquet:   0%|          | 0.00/174k [00:00<?, ?B/s]

python_train-00000-of-00001.parquet:   0%|          | 0.00/126k [00:00<?, ?B/s]

...

Generating java_test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Generating python_train split:   0%|          | 0/1884 [00:00<?, ? examples/s]

Generating python_test split:   0%|          | 0/406 [00:00<?, ? examples/s]

Generating pharo_train split:   0%|          | 0/1298 [00:00<?, ? examples/s]

Generating pharo_test split:   0%|          | 0/289 [00:00<?, ? examples/s]

In [9]:
# model_name = MODEL_NAME.format(alias, strategy, batch_size, num_epochs, num_iterations, head)
model_name = MODEL_NAME.format(alias, num_iterations, head)
print("Model to evaluate:", model_name)

Model to evaluate: nb-i0-hNONE


In [10]:
langs = ["java", "python", "pharo"]

labels = {
    "java": ["summary", "Ownership", "Expand", "usage", "Pointer", "deprecation", "rational"],
    "python": ["Usage", "Parameters", "DevelopmentNotes", "Expand", "Summary"],
    "pharo": ["Keyimplementationpoints", "Example", "Responsibilities", "Classreferences", "Intent", "Keymessages", "Collaborators"]
}

java_df = []
python_df = []
pharo_df = []

lang = "java"
for s in ["train", "test"]:
	df = pd.DataFrame(dataset[f"{lang}_{s}"])
	labels_df = pd.DataFrame(df["labels"].tolist(), columns=labels[lang])
	df = pd.concat([df, labels_df], axis=1)
	java_df.append(df)
java_df = pd.concat(java_df).reset_index(drop=True)

lang = "python"
for s in ["train", "test"]:
	df = pd.DataFrame(dataset[f"{lang}_{s}"])
	labels_df = pd.DataFrame(df["labels"].tolist(), columns=labels[lang])
	df = pd.concat([df, labels_df], axis=1)
	python_df.append(df)
python_df = pd.concat(python_df).reset_index(drop=True)

lang = "pharo"
for s in ["train", "test"]:
	df = pd.DataFrame(dataset[f"{lang}_{s}"])
	labels_df = pd.DataFrame(df["labels"].tolist(), columns=labels[lang])
	df = pd.concat([df, labels_df], axis=1)
	pharo_df.append(df)
pharo_df = pd.concat(pharo_df).reset_index(drop=True)

In [18]:
# Train Naive Bayes Model
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

models = {}

for lan in langs:
    model = Pipeline([
        ("vectorizer", CountVectorizer()),
        ("classifier", MultinomialNB())
    ])

    if lan=='java':
        y_true_train = java_df[java_df["partition"] == 0][labels[lan]].apply(lambda x: x.argmax(), axis=1)
    elif lan=='python':
        y_true_train = python_df[python_df["partition"] == 0][labels[lan]].apply(lambda x: x.argmax(), axis=1)
    elif lan=='pharo':
        y_true_train = pharo_df[pharo_df["partition"] == 0][labels[lan]].apply(lambda x: x.argmax(), axis=1)
    models[lan] = model.fit(dataset[f"{lan}_train"]["combo"], y_true_train)

In [19]:
total_flops = 0
total_time = 0
scores = []

for lan in langs:

    # Starting the profiler as context
    with torch.profiler.profile(with_flops=True) as p:
        begin = time.time()

        # Running multiple inference experiments
        for i in range(NUM_EXPERIMENTS):
            # Calling the model
            # y_pred = model(dataset[f"{lan}_test"]["combo"]).cpu().numpy().T
            # create vector with majority class
            #y_pred = np.zeros((len(dataset[f"{lan}_test"]["combo"]), len(labels[lan])))
            #y_pred = np.zeros((len(labels[lan]), len(dataset[f"{lan}_test"]["combo"])))
            #y_pred[majority_label_java,:] = 1
            preds = models[lan].predict(dataset[f"{lan}_test"]["combo"])
            y_pred = np.eye(len(labels[lan]))[preds]
            y_pred = y_pred.transpose()

        total = time.time() - begin
        total_time = total_time + total

    # Calculating the total of FLOPs used
    total_flops = total_flops + (sum(k.flops for k in p.key_averages()) / 1e9)

    # Getting the ground truth
    y_true = np.array(dataset[f"{lan}_test"]["labels"]).T

    for i in range(len(y_pred)):
        assert(len(y_pred[i]) == len(y_true[i]))

        # Calculating confusion matrix
        tp = sum([true == pred == 1 for (true, pred) in zip(y_true[i], y_pred[i])])
        tn = sum([true == pred == 0 for (true, pred) in zip(y_true[i], y_pred[i])])
        fp = sum([true == 0 and pred == 1 for (true, pred) in zip(y_true[i], y_pred[i])])
        fn = sum([true == 1 and pred == 0 for (true, pred) in zip(y_true[i], y_pred[i])])

        # Calculating error metrics
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1 = (2*tp) / (2*tp + fp + fn)

        scores.append({
            "lan": lan,
            "cat": labels[lan][i],
            "precision": precision,
            "recall": recall,
            "f1": f1
        })

scores = pd.DataFrame(scores)

print("Compute in GFLOPs:", total_flops / NUM_EXPERIMENTS)
print("Avg. runtime in seconds:", total_time / NUM_EXPERIMENTS)

Compute in GFLOPs: 0.0
Avg. runtime in seconds: 0.039604806900024415


<ipython-input-19-ae7ea20c826e>:42: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)


In [13]:
scores

,lan,cat,precision,recall,f1
0,java,summary,0.735354,0.816143,0.773645
1,java,Ownership,0.977273,0.955556,0.966292
2,java,Expand,0.304348,0.137255,0.189189
3,java,usage,0.698113,0.686775,0.692398
4,java,Pointer,0.752577,0.793478,0.772487
5,java,deprecation,1.000000,0.133333,0.235294
6,java,rational,0.200000,0.073529,0.107527
7,python,Usage,0.647059,0.545455,0.591928
8,python,Parameters,0.645349,0.867188,0.740000
9,python,DevelopmentNotes,0.466667,0.170732,0.250000


In [20]:
max_avg_runtime = 5
max_avg_flops = 5000

# s𝑢𝑏𝑚𝑖𝑠𝑠𝑖𝑜𝑛_𝑠𝑐𝑜𝑟𝑒(𝑚𝑜𝑑𝑒𝑙)=(𝑎𝑣𝑔. 𝐹1)×0.60+((𝑚𝑎𝑥_𝑎𝑣𝑔_𝑟𝑢𝑛𝑡𝑖𝑚𝑒−𝑚𝑒𝑎𝑠𝑢𝑟𝑒𝑑_𝑎𝑣𝑔_𝑟𝑢𝑛𝑡𝑖𝑚𝑒)/𝑚𝑎𝑥_𝑎𝑣𝑔_𝑟𝑢𝑛𝑡𝑖𝑚𝑒)×0.2+((𝑚𝑎𝑥_GFLOPs−𝑚𝑒𝑎𝑠𝑢𝑟𝑒𝑑_GFLOPs)/𝑚𝑎𝑥_GFLOPs)×0.2
def score(avg_f1, avg_runtime, avg_flops):
    return (0.6 * avg_f1 +
      0.2 * ((max_avg_runtime - avg_runtime) / max_avg_runtime) +
      0.2 * ((max_avg_flops - avg_flops) / max_avg_flops))

avg_f1 = float(scores.f1.mean())
avg_runtime = total_time / NUM_EXPERIMENTS
avg_flops = total_flops / NUM_EXPERIMENTS

submission_score = score(avg_f1, avg_runtime, avg_flops)

print("Submission score:", round(submission_score, 3))

Submission score: 0.597


In [17]:
scoring_details = {}

for i, row in scores.iterrows():
	scoring_details[f"{row['lan']}_{ row['cat']}_f1"] = row["f1"]

scoring_details = {
    **scoring_details,
    **scores.groupby("lan").f1.mean().add_suffix("_avg_f1").to_dict()
}

scoring_details["avg_f1"] = avg_f1
scoring_details["avg_runtime"] = avg_runtime
scoring_details["avg_flops"] = avg_flops
scoring_details["submission_score"] = submission_score

scoring_details

{'java_summary_f1': 0.7736450584484591,
 'java_Ownership_f1': 0.9662921348314607,
 'java_Expand_f1': 0.1891891891891892,
 'java_usage_f1': 0.6923976608187135,
 'java_Pointer_f1': 0.7724867724867724,
 'java_deprecation_f1': 0.23529411764705882,
 'java_rational_f1': 0.10752688172043011,
 'python_Usage_f1': 0.5919282511210763,
 'python_Parameters_f1': 0.74,
 'python_DevelopmentNotes_f1': 0.25,
 'python_Expand_f1': 0.3783783783783784,
 'python_Summary_f1': 0.5657894736842105,
 'pharo_Keyimplementationpoints_f1': 0.46808510638297873,
 'pharo_Example_f1': 0.8145161290322581,
 'pharo_Responsibilities_f1': 0.5172413793103449,
 'pharo_Classreferences_f1': 0.0,
 'pharo_Intent_f1': 0.4,
 'pharo_Keymessages_f1': 0.5384615384615384,
 'pharo_Collaborators_f1': 0.0,
 'java_avg_f1': 0.5338331164488691,
 'pharo_avg_f1': 0.39118630759816003,
 'python_avg_f1': 0.505219220636733,
 'avg_f1': 0.4737490563954142,
 'avg_runtime': 0.03600811958312988,
 'avg_flops': 0.0,
 'submission_score': 0.6828091090539233}